<a href="https://colab.research.google.com/github/vitamingyu/NLP-LLM/blob/main/tf_38naver_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 49.2 MB/s eta 0:00:00


In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
import tensorflow as tf

In [12]:
# urllib.request.urlretrieve("https://raw.githubusercontent.com/pykwon/python/master/testdata_utf8/ratings_train.txt", filename='ratings_train.txt')
train_data = pd.read_table('https://raw.githubusercontent.com/pykwon/python/master/testdata_utf8/ratings_train.txt')
test_data = pd.read_table('https://raw.githubusercontent.com/pykwon/python/master/testdata_utf8/ratings_test.txt')
print(train_data[:2], train_data.shape)  # (150000, 3)
print(test_data[:2], test_data.shape)  # (50000, 3)

# data cleaning
print(train_data['document'].nunique(), test_data['document'].nunique())  # 146182 49157  중복자료가 있네
train_data.drop_duplicates(subset=['document'], inplace=True)
print('샘플 수 : ', len(train_data))
print(train_data.groupby('label').size().reset_index(name='count'))

print(train_data.isnull().values.any())
train_data = train_data.dropna(how='any')
print(train_data.shape)

# 한글 자룜나 허용
train_data['document'] = train_data['document'].str.replace('[^가-힣 ]','')  # []안에 ^은 부정. 가~힣이 아니고 공백이 문장중간에 있으면 지워
print(train_data[:3])

# white space를 empty value로 변경
train_data['document'] = train_data['document'].str.replace('^ +', '')  # 바로 ^로하면 시작부분. 시작글자가 공백이면 제거
train_data['document'].replace('', np.nan, inplace=True)  # empty value를 Null처리
print(train_data.isnull().sum())
print(train_data.loc[train_data.document.isnull()][:3])

train_data = train_data.dropna(how='any')
print(train_data.shape)  # (145663, 3)

# TEST
test_data.drop_duplicates(subset=['document'], inplace=True)
test_data['document'] = test_data['document'].str.replace('[^가-힣 ]','')
test_data['document'] = test_data['document'].str.replace('^ +', '')
test_data['document'].replace('', np.nan, inplace=True)
test_data = test_data.dropna(how='any')
print(test_data.shape)

        id                           document  label
0  9976970                아 더빙.. 진짜 짜증나네요 목소리      0
1  3819312  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1 (150000, 3)
        id              document  label
0  6270596                   굳 ㅋ      1
1  9274899  GDNTOPCLASSINTHECLUB      0 (50000, 3)
146182 49157
샘플 수 :  146183
   label  count
0      0  73342
1      1  72841
True
(146182, 3)


<ipython-input-12-7ec4163383aa>:18: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('[^가-힣 ]','')  # []안에 ^은 부정. 가~힣이 아니고 공백이 문장중간에 있으면 지워


         id                    document  label
0   9976970           아 더빙 진짜 짜증나네요 목소리      0
1   3819312  흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나      1
2  10265843           너무재밓었다그래서보는것을추천한다      0
id            0
document    934
label         0
dtype: int64
           id document  label
404   4221289      NaN      0
412   9509970      NaN      1
470  10147571      NaN      1


<ipython-input-12-7ec4163383aa>:22: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('^ +', '')  # 바로 ^로하면 시작부분. 시작글자가 공백이면 제거


(145248, 3)
(48779, 3)


<ipython-input-12-7ec4163383aa>:32: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace('[^가-힣 ]','')
<ipython-input-12-7ec4163383aa>:33: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace('^ +', '')


In [13]:
# 불용어
stopwords = ['의', '를', '을', '가', '들', '좀', '잘', '하여', '으로', '에', '며', '한', '와', '과', '리', '더' ,'에는', '하다', '아', '']

from tqdm import tqdm
okt = Okt()
x_train = []
for sentence in tqdm(train_data['document']):
  tokenized_sentence = okt.morphs(sentence, stem=True)
  stop_remove_sentence = [word for word in tokenized_sentence if not word in stopwords]  # 불용어 제거
  x_train.append(stop_remove_sentence)

print(x_train[:3])

x_test = []
for sentence in tqdm(test_data['document']):
  tokenized_sentence = okt.morphs(sentence, stem=True)
  stop_remove_sentence = [word for word in tokenized_sentence if not word in stopwords]  # 불용어 제거
  x_test.append(stop_remove_sentence)

print(x_test[:3])

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
print(tokenizer.word_index)

100%|██████████| 145248/145248 [13:31<00:00, 179.00it/s]


[['더빙', '진짜', '짜증나다', '목소리'], ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'], ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다']]


100%|██████████| 48779/48779 [04:49<00:00, 168.48it/s]


[['굳다'], ['뭐', '야', '이', '평점', '은', '나쁘다', '않다', '점', '짜다', '는', '더욱', '아니다'], ['지루하다', '않다', '완전', '막장', '임', '돈', '주다', '보기']]
{'이': 1, '영화': 2, '보다': 3, '도': 4, '는': 5, '은': 6, '없다': 7, '이다': 8, '있다': 9, '좋다': 10, '너무': 11, '다': 12, '정말': 13, '되다': 14, '재밌다': 15, '적': 16, '만': 17, '같다': 18, '진짜': 19, '로': 20, '아니다': 21, '않다': 22, '점': 23, '에서': 24, '만들다': 25, '나오다': 26, '연기': 27, '것': 28, '평점': 29, '내': 30, '최고': 31, '그': 32, '나': 33, '안': 34, '인': 35, '스토리': 36, '생각': 37, '못': 38, '왜': 39, '드라마': 40, '게': 41, '감동': 42, '사람': 43, '보고': 44, '이렇다': 45, '말': 46, '고': 47, '아깝다': 48, '배우': 49, '때': 50, '감독': 51, '거': 52, '그냥': 53, '재미있다': 54, '요': 55, '재미': 56, '시간': 57, '내용': 58, '뭐': 59, '까지': 60, '중': 61, '주다': 62, '자다': 63, '하고': 64, '지루하다': 65, '재미없다': 66, '네': 67, '쓰레기': 68, '수': 69, '모르다': 70, '가다': 71, '들다': 72, '그렇다': 73, '싶다': 74, '지': 75, '작품': 76, '사랑': 77, '알다': 78, '하나': 79, '다시': 80, '마지막': 81, '볼': 82, '이건': 83, '정도': 84, '저': 85, '완전': 86, '오다': 87, '많다': 88, '처음': 89, '

In [14]:
# 등장빈도가 3회 보다 적은 단어들 비중 확인
threshold = 3
total_cnt = len(tokenizer.word_index)
rare_cnt = 0  # 등장빈도가 3회 보다 적은 단어들 건수
total_freq = 0  # 전체 단어 빈도수 - 훈련 데이터에 대한
rare_freq = 0  # 등장빈도가 3회 보다 적은 단어들 비율

# 단어와 빈도수 k:v로 처리
for k, v in tokenizer.word_counts.items():
  total_freq = total_freq + v
  if v < threshold:
    rare_cnt = rare_cnt + 1
    rare_freq = rare_freq + v

print('total_cnt : ', total_cnt)
print('rare_cnt : ', rare_cnt)
print('3회 미만 단어 비율 : ', (rare_cnt / total_cnt) * 100)
print('전체 빈도에서 3회 미만 빈도 비율 : ', (rare_freq / total_freq) * 100)

# 전체 단어 수 중에서 빈도 수 2이하인 단어는 제거
vocab_size = total_cnt - rare_cnt + 1
print('vocab_size : ', vocab_size)

total_cnt :  43070
rare_cnt :  23850
3회 미만 단어 비율 :  55.37497097747852
전체 빈도에서 3회 미만 빈도 비율 :  1.7771260215491445
vocab_size :  19221


In [15]:
# vocab_size : 19219
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)
print(x_train[:3])

y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

# empty smaple 제거 : 빈도수가 낮은 단어가 삭제됨으로 해서 빈도수가 낮은 단어로 구성된 샘플들은 empty sample이 됨
# 그러므로 제거작업이 필요
drop_train = [index for index, sentence in enumerate(x_train) if len(sentence) < 1]
print(drop_train)

x_train = np.delete(x_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(x_train), len(y_train))

# padding
print('리뷰 최대 길이 : ', max(len(review) for review in x_train))
print('리뷰 평균 길이 : ', sum(map(len, x_train)) / len(x_train))

max_len = 30  # padding의 요소 수는 대략 30을 부여
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

[[447, 19, 254, 651], [920, 449, 44, 594, 2, 209, 1427, 27, 943, 666, 22], [378, 2404, 2224, 5608, 3, 217, 12]]
[28, 151, 319, 414, 1148, 1346, 1571, 1710, 2319, 2343, 2430, 3207, 3435, 3436, 3646, 4284, 4605, 4802, 5014, 5622, 5715, 7130, 7778, 8243, 8269, 9647, 9881, 10583, 10619, 10960, 12454, 12704, 12877, 13715, 13769, 15406, 15418, 16760, 18504, 18574, 18603, 19263, 20515, 21273, 22078, 22349, 24547, 24835, 24995, 25374, 25824, 26276, 26816, 27071, 27458, 28433, 28895, 29316, 29552, 29574, 32171, 32415, 32827, 32838, 33654, 33970, 34168, 34317, 34513, 37047, 37427, 39953, 40307, 40608, 40930, 43146, 43251, 43349, 43860, 43922, 44032, 44567, 45326, 46296, 46725, 47392, 47801, 48140, 48762, 50950, 51028, 51874, 51977, 52712, 52914, 53498, 54573, 54746, 54808, 56205, 56901, 56966, 57312, 57320, 58859, 59373, 60349, 60367, 60646, 61056, 61313, 61556, 61711, 61876, 62033, 62518, 63412, 63672, 64156, 64256, 65108, 65405, 65973, 66589, 67619, 67867, 67923, 67970, 69399, 69647, 70430, 70

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:5071: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


In [16]:
# model
from keras.layers import LSTM, Embedding, Dense
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_size, 100))  # 차원수는 100차원 줌
model.add(LSTM(128, activation='tanh'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

es = EarlyStopping(monitor='val_loss', mode='auto', patience=5)
mc = ModelCheckpoint('tf38.hdf5', monitor='val_loss', save_best_only=True)

history = model.fit(x_train, y_train, validation_split=0.2, batch_size=64, callbacks=[es, mc], verbose=2)

print('test 정확도 : ', model.evaluate(x_test, y_test)[1])
print('test 손실도 : ', model.evaluate(x_test, y_test)[0])

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1813/1813 - 59s - loss: 0.3861 - acc: 0.8226 - val_loss: 0.3469 - val_acc: 0.8476 - 59s/epoch - 33ms/step
1525/1525 [==============================] - 5s 3ms/step - loss: 0.3497 - acc: 0.8450
test 정확도 :  0.8449742794036865
1525/1525 [==============================] - 6s 4ms/step - loss: 0.3497 - acc: 0.8450
test 손실도 :  0.34973442554473877


In [17]:
mymodel = load_model('tf38.hdf5')

# 리뷰 예측
def predictFunc(new_sen):
  new_sen = re.sub(r'[^가-힣 ]', '', new_sen)  # 가~힣, 공백을 제거한 나머지만 남김
  new_sen = okt.morphs(new_sen, stem=True)  # 어간처리해줌
  new_sen = [word for word in new_sen if not word in stopwords]
  encoded = tokenizer.texts_to_sequences([new_sen])
  pad_new = pad_sequences(encoded, maxlen=max_len)
  score = float(mymodel.predict(pad_new))
  if score > 0.5:
    print('{:.2f}% 확률로 긍정'.format(score * 100))
  else:
    print('{:.2f}% 확률로 부정'.format(score * 100))

predictFunc('원작의 긴장감을 제대로 살려내지 못했다')
predictFunc('액션이 없는데도 재미 있는 몇안되는 영화')
predictFunc('보면서 웃지 않으면 환불')
predictFunc('엄청나게 재미없는 영화')
predictFunc('이 영화 맛도리다')

1/1 [==============================] - 0s 391ms/step
22.11% 확률로 부정
1/1 [==============================] - 0s 17ms/step
29.92% 확률로 부정
1/1 [==============================] - 0s 19ms/step
13.68% 확률로 부정
1/1 [==============================] - 0s 19ms/step
2.65% 확률로 부정
1/1 [==============================] - 0s 24ms/step
49.46% 확률로 부정
